In [1]:
import pandas as pd
import numpy as np

In [2]:
netflixdf= pd.read_csv('C:/Users/Ankita Mishra/Desktop/sql/netflix_titles/netflix_titles.csv')
netflixdf.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,9/25/2021,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,9/24/2021,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,9/24/2021,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,9/24/2021,2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,9/24/2021,2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [3]:
netflixdf.shape
netflixdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


# Data Cleaning

First we check for duplicated values and drop any duplicated columns. The find the number of nulls in each column.

In [4]:
# check for duplicates and nulls
netflixdf.duplicated().value_counts()
netflixdf.isnull().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

The director column has a lot of nulls so we attempt to populate it by finding a relationship between the director column and cast column.


In [5]:
netflixdf['director_cast'] = netflixdf['director'] + '---'+ netflixdf['cast']

# Count the occurrences of each 'director_cast' combination
director_cast_counts = netflixdf['director_cast'].value_counts()

# Filter the combinations with more than one occurrence
director_cast_counts = director_cast_counts[director_cast_counts > 1]

for director_cast, director_cast_count in director_cast_counts.iteritems():
    director, cast = director_cast.split('---')
    netflixdf.loc[(netflixdf['director'].isnull()) & (netflixdf['cast'] == cast), 'director'] = director

# Fill the remaining NULL values in 'director' with 'Not Given'
netflixdf['director'].fillna('Not Given', inplace=True)


Finding a similar relationship between director and country to populate the nulls in the country column.

In [6]:
netflixdf['director_country'] = netflixdf['director'] + '---'+ netflixdf['country']
director_country_counts = netflixdf['director_country'].value_counts()
director_cast_counts = director_cast_counts[director_cast_counts > 1]

for director_country, director_country_count in director_country_counts.iteritems():
    director, country = director_country.split('---')
    netflixdf.loc[(netflixdf['country'].isnull()) & (netflixdf['director'] == director), 'country'] = country
    
netflixdf['country'].fillna('Not Given', inplace=True)

netflixdf.drop(columns=['director_cast', 'director_country'], inplace=True)
netflixdf.head()


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,9/25/2021,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,Not Given,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,9/24/2021,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",France,9/24/2021,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,Not Given,NaN,United States,9/24/2021,2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,Not Given,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,9/24/2021,2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


date_added, rating and duration have an insignificant number of nulls so those rows can be dropped from the dataframe without causing much difference to our analysis.

In [7]:
null_rows= netflixdf[(netflixdf['date_added'].isnull())|
                    (netflixdf['rating'].isnull())|
                    (netflixdf['duration'].isnull())]
print(null_rows)

     show_id     type                                              title  \
5541   s5542    Movie                                    Louis C.K. 2017   
5794   s5795    Movie                              Louis C.K.: Hilarious   
5813   s5814    Movie               Louis C.K.: Live at the Comedy Store   
5989   s5990    Movie  13TH: A Conversation with Oprah Winfrey & Ava ...   
6066   s6067  TV Show        A Young Doctor's Notebook and Other Stories   
6174   s6175  TV Show                    Anthony Bourdain: Parts Unknown   
6795   s6796  TV Show                                            Frasier   
6806   s6807  TV Show                                            Friends   
6827   s6828  TV Show                  Gargantia on the Verdurous Planet   
6901   s6902  TV Show                                    Gunslinger Girl   
7196   s7197  TV Show                                           Kikoriki   
7254   s7255  TV Show                                La Familia P. Luche   
7312   s7313

In [8]:
netflixdf = netflixdf.dropna(subset=['date_added', 'rating', 'duration'])

Check for duplicates and nulls and make sure each column has same number of records after the cleaning. Drop columns that we would not need for our analysis and visualization.

In [9]:
# check for duplicates and nulls
netflixdf.duplicated().value_counts()
netflixdf.isnull().sum()

show_id           0
type              0
title             0
director          0
cast            825
country           0
date_added        0
release_year      0
rating            0
duration          0
listed_in         0
description       0
dtype: int64

In [10]:
netflixdf= netflixdf.drop(columns=['cast', 'description'])

In [11]:
netflixdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8790 entries, 0 to 8806
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8790 non-null   object
 1   type          8790 non-null   object
 2   title         8790 non-null   object
 3   director      8790 non-null   object
 4   country       8790 non-null   object
 5   date_added    8790 non-null   object
 6   release_year  8790 non-null   int64 
 7   rating        8790 non-null   object
 8   duration      8790 non-null   object
 9   listed_in     8790 non-null   object
dtypes: int64(1), object(9)
memory usage: 755.4+ KB


To make visualization easier, remove any additional countries after the first country in the country column.

In [13]:
split_country = netflixdf['country'].str.split(',', expand=True)

# Create a new column 'country1' with the first split country
netflixdf['country1'] = split_country[0]
netflixdf.drop(columns=['country'], inplace=True)
netflixdf.rename(columns={'country1': 'country'}, inplace=True)

In [18]:
clean_netflixdf= netflixdf
clean_netflixdf.to_csv('netflix_titles_filtered.csv')

# Data Summary

In [16]:
cols = ["type", "country", "rating"]

for col in cols:
    print(netflixdf[col].value_counts(), "\n")
netflixdf["type"].value_counts(normalize=True)

Movie      6126
TV Show    2664
Name: type, dtype: int64 

United States     3639
India             1066
United Kingdom     634
Canada             273
Not Given          272
                  ... 
Syria                1
Namibia              1
Senegal              1
Luxembourg           1
Somalia              1
Name: country, Length: 87, dtype: int64 

TV-MA       3205
TV-14       2157
TV-PG        861
R            799
PG-13        490
TV-Y7        333
TV-Y         306
PG           287
TV-G         220
NR            79
G             41
TV-Y7-FV       6
NC-17          3
UR             3
Name: rating, dtype: int64 



Movie      0.696928
TV Show    0.303072
Name: type, dtype: float64

There are 86 unique countries and 272 records with the country Not Given, and 14 types of ratings with most them them being TV shows for mature adults (TV-MA). 